In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import io
import numpy as np


def create_sequences(text, sequence_length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - sequence_length, step):
        sequences.append(text[i: i + sequence_length])
        next_chars.append(text[i + sequence_length])
    return sequences, next_chars


def build_model(sequence_length, chars):
    model = Sequential()
    model.add(LSTM(128, input_shape=(sequence_length, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model


def sample(preds, temperature=1.0):

    if temperature == 0:
        temperature = 1

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def extract_characters(text):
    return sorted(list(set(text)))


def get_chars_index_dicts(chars):
    return dict((c, i) for i, c in enumerate(chars)), dict((i, c) for i, c in enumerate(chars))


def read_corpus(path):
    with io.open(path, 'r', encoding='utf8') as f:
        return f.read().lower()


def vectorize(sequences, sequence_length, chars, char_to_index, next_chars):
    X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool)
    y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sequences):
        for t, char in enumerate(sentence):
            X[i, t, char_to_index[char]] = 1
        y[i, char_to_index[next_chars[i]]] = 1

    return X, y

**  LIBRARIES**
Put simply, the __future__ statement forces Python interpreters to use newer features of the language.
NumPy is a scientific and numerical computing extension to the Python programming language.
 Python defines a set of functions that are used to generate or manipulate random numbers.
  System-specific parameters and functions. This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.
  If you only need to save the architecture of a model, and not its weights or its training configuration

 ---------------------------------------------- 
 
 
  

In [0]:
from __future__ import print_function    
import numpy as np
import random
import sys
from keras.models import load_model



SEQUENCE_LENGTH = 40
SEQUENCE_STEP = 2

EPOCHS10 = 2
DIVERSITY = 1.0



#Read the corpus and get unique characters from the corpus.
text = read_corpus("jazz.txt")
chars = extract_characters(text)

#Create sequences that will be used as the input to the network.
#Create next_chars array that will serve as the labels during the training.
sequences, next_chars = create_sequences(text, SEQUENCE_LENGTH, SEQUENCE_STEP)
char_to_index, indices_char = get_chars_index_dicts(chars)

#The network is not able to work with characters and strings, we need to vectorise.
X, y = vectorize(sequences, SEQUENCE_LENGTH, chars, char_to_index, next_chars)

#Define the structure of the model.
model = build_model(SEQUENCE_LENGTH, chars)

#Train the model

model.fit(X, y, batch_size=128, epochs=EPOCHS10)


model.save('jazz.h5')

#model.save_weights("final.h5")

#model = load_model("final.h5")  # you can skip training by loading the trained weights



Epoch 1/2
351522/351522 [==============================] - 240s 681us/step - loss: 1.8359
Epoch 2/2
351522/351522 [==============================] - 243s 692us/step - loss: 1.5730


In [0]:

model = load_model('jazz.h5')
#Pick a random sequence and make the network continue
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    # insert your 40-chars long string. OBS it needs to be exactly 40 chars!
    sentence = "You are my love and my life, girl, Your "
    sentence = sentence.lower() #stop here-------------^"
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_to_index[char]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "you are my love and my life, girl, your "
you are my love and my life, girl, your lips the moonlight
i was a spin the stars
and so when i say i love it all the blues

i wish a spark of my heart when you say
i was a said a spring and the star

i won't be the way the stars

i was a little said to me

i was a swell and when i were the moonlight

i was a spin the stars of my heart

i will i love you the way the same

when i was i were a swell and start

i was a same is the stars

i

----- diversity: 0.5
----- Generating with seed: "you are my love and my life, girl, your "
you are my love and my life, girl, your sun
i cat all the moonlight and been the stard


you have all the stars of a little stars
the moonlight is a swings that i start
but i can meet the night and stay 
when i list my heart we crieves
he long of my love

it always should like is to the star

of you and what is someone i find the stars

what he gave i'm to a word

i don'